In [1]:
import numpy as np
import cv2

from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt

from os import listdir
import cv2
from mtcnn.mtcnn import MTCNN
detector=MTCNN()

Using TensorFlow backend.


In [2]:
color = (67,67,67)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    #preprocess_input normalizes input in scale of [-1, +1]. You must apply same normalization in prediction.
    #Ref: https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py (Line 45)
    img = preprocess_input(img)
    return img

In [3]:
def loadVggFaceModel():
	model = Sequential()
	model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
	model.add(Convolution2D(64, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(64, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(128, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(128, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(256, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(256, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(256, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(Convolution2D(4096, (7, 7), activation='relu'))
	model.add(Dropout(0.5))
	model.add(Convolution2D(4096, (1, 1), activation='relu'))
	model.add(Dropout(0.5))
	model.add(Convolution2D(2622, (1, 1)))
	model.add(Flatten())
	model.add(Activation('softmax'))
	
	#you can download pretrained weights from https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
	from keras.models import model_from_json
	model.load_weights('vgg_face_weights.h5')
	
	vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
	
	return vgg_face_descriptor

In [4]:
model = loadVggFaceModel()

In [5]:
person_images = "D:/data/oneshot/"

In [6]:
persons = dict()

for file in listdir(person_images):
	person, extension = file.split(".")
	persons[person] = model.predict(preprocess_image('D:/data/oneshot/%s.jpg' % (person)))[0,:]

In [7]:
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [8]:
from datetime import datetime
# Capture time duration of faces detected
faceFound = {}
ts = [0,0,False,' ']

In [9]:
import urllib
from urllib import request as ur

In [21]:
url='http://192.168.1.225:8080/out.jpg?q=30&id=0.5380787588953155&r=1579594544773'
while True:
    imgresp=ur.urlopen(url)
    imgNp=np.array(bytearray(imgresp.read()),dtype=np.uint8)
    img=cv2.imdecode(imgNp,-1)
    #_,img=video_capture.read()

    result=detector.detect_faces(img)
# in the result we will have multiple number of dictionaries for each image, and we need to extract the bounding box of the face.      
    if len(result):
        if ts[2] == False:
            ts[0] = datetime.now()
            ts[2] = True
        for x in result:
            #print (result)
            bounding_box=x['box']

            #after extracting bounding box, draw rectangle around the face.

            cv2.rectangle(img,(bounding_box),(0,0,255),3)

            #save the image.

            x1,y3,w,h=x['box']
            x2,y2=x1+w,y3+h
            cropped=img[y3:y2, x1:x2]
            cropped = cv2.resize(cropped, (224, 224))
            img_pixels = image.img_to_array(cropped)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
    #img_pixels /= 255
    #employee dictionary is using preprocess_image and it normalizes in scale of [-1, +1]
            img_pixels /= 127.5
            img_pixels -= 1

            captured_representation = model.predict(img_pixels)[0,:]
            #print(captured_representation)

            found = 0
            for i in persons:
                person_name = i
                representation = persons[i]
                similarity = findCosineSimilarity(representation, captured_representation)
                if(similarity < 0.28):
                    cv2.putText(img,person_name,(x1,y3-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
                    ts[3] = person_name

                    found = 1
                    break

    #connect face and text
            #cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
            #cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)

            if(found == 0): #if found image is not in employee database
                cv2.putText(img,' ',(x1,y3-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
    else:
        if ts[2] == True:
            ts[1] = datetime.now()
            ts[2] = False
            diff = ts[1]-ts[0]
            if diff.seconds!=0:
                faceFound.update({ts[3]:[diff.seconds+1, ts[0].strftime("%m/%d/%Y %H:%M:%S"), ts[1].strftime("%m/%d/%Y %H:%M:%S")]})


    #cv2.imshow('img',img)

    if cv2.waitKey(25) & 0xFF == ord('q'): #press q to quit
        break

    #kill open cv things		
    #cap.release()
cv2.destroyAllWindows()

In [27]:
time_log=pd.DataFrame(faceFound,index=('total time(sec)','entry time','exit time'))

In [28]:
import pandas as pd

In [29]:
time_log

,shanu,unique
total time(sec),3,7
entry time,01/21/2020 15:14:17,01/21/2020 15:14:10
exit time,01/21/2020 15:14:20,01/21/2020 15:14:16
